# TODO
Properly comment the code

In [1]:
import pandas as pd

In [3]:
input_df = pd.read_csv('../data/web_input.csv')

In [5]:
input_df.head()

,id,type,integerId,name,riskLabel,cityAndRegion,country
0,http://data.allim.com/core/Integer#intent018lUW,http://data.allim.com/core#Entity,intent018lUW,Aral Southern Xinjiang Hongguopin Famers Profe...,http://data.allim.com/core#CnForcedLaborl,"Alar, Xinjiang",China
1,http://data.allim.com/core/Integer#intent01F6Yz,http://data.allim.com/core#Entity,intent01F6Yz,"Wusu Yuanda Cotton Industry Co., Ltd.",http://data.allim.com/core#CnForcedLabor,"Wusu, Tacheng, Xinjiang",China
2,http://data.allim.com/core/Integer#intent01F6Yz,http://data.allim.com/core#Vessel,intent01F6Yz,"Wusu Yuanda Cotton Industry Co., Ltd.",http://data.allim.com/core#CnForcedLabor,"Wusu, Tacheng, Xinjiang",China
3,http://data.allim.com/core/Integer#intent01GiDN,http://data.allim.com/core#Entity,intent01GiDN,"Shihezi Tiancheng Water Saving Equipment Co., ...",http://data.allim.com/core#CnForcedLaborl,NaN,China
4,http://data.allim.com/core/Integer#intent01mMB4,http://data.allim.com/core#Entity,intent01mMB4,"Xinjiang Xinte Crystal Silicon Hightech Co., Ltd.",http://data.allim.com/core#CnForcedLabor0l,"Urumqi, Xinjiang",China


In [14]:
input_df['Date Generated'] = '2022-07-14 14:35:04'
input_df['AllimID'] = input_df['integerId']
input_df['Name'] = input_df['name']

In [172]:
def get_type_from_url(row):
    type_value = row['type'].split("#")[1]
    return f'{type_value}'

In [173]:
input_df['Entity Type'] = input_df.apply(get_type_from_url, axis=1)

In [210]:
def get_risk_label_from_input(row):
    risk_value = row['riskLabel'].split("#")[1]

    if 'CnForcedLabor' in risk_value:
        return 'CnForcedLabor'

    return f'{risk_value}'

In [211]:
input_df['Risk Labels'] = input_df.apply(get_risk_label_from_input, axis=1)

In [36]:
input_df['Primary City'] = input_df['cityAndRegion'].fillna('')

In [38]:
input_df['Primary Country'] = input_df['country']

In [40]:
input_df.dtypes

id                 object
type               object
integerId          object
name               object
riskLabel          object
cityAndRegion      object
country            object
Date Generated     object
AllimID            object
Name               object
Entity Type        object
Risk Labels        object
Primary City       object
Primary Country    object
dtype: object

In [212]:
final_df = input_df.drop(['id', 'type', 'integerId', 'name', 'riskLabel', 'cityAndRegion', 'country'], axis=1)

In [213]:
final_df.head(15)

,Date Generated,AllimID,Name,Entity Type,Risk Labels,Primary City,Primary Country
0,2022-07-14 14:35:04,intent018lUW,Aral Southern Xinjiang Hongguopin Famers Profe...,Entity,CnForcedLabor,"Alar, Xinjiang",China
1,2022-07-14 14:35:04,intent01F6Yz,"Wusu Yuanda Cotton Industry Co., Ltd.",Entity,CnForcedLabor,"Wusu, Tacheng, Xinjiang",China
2,2022-07-14 14:35:04,intent01F6Yz,"Wusu Yuanda Cotton Industry Co., Ltd.",Vessel,CnForcedLabor,"Wusu, Tacheng, Xinjiang",China
3,2022-07-14 14:35:04,intent01GiDN,"Shihezi Tiancheng Water Saving Equipment Co., ...",Entity,CnForcedLabor,,China
4,2022-07-14 14:35:04,intent01mMB4,"Xinjiang Xinte Crystal Silicon Hightech Co., Ltd.",Entity,CnForcedLabor,"Urumqi, Xinjiang",China
5,2022-07-14 14:35:04,intent01sKCr,"Shihezi West Ningxin Carbon Co., Ltd.",Entity,CnForcedLabor,"Shihezi, Xinjiang",China
6,2022-07-14 14:35:04,intent02arS7,"Yengisar Tianyan Industry Co., Ltd.",Entity,CnForcedLabor,"Yengisar County, Kashgar Prefecture, Xinjiang",China
7,2022-07-14 14:35:04,intent02CuQE,Hunan Xinxiang Expressway Construction Develop...,Entity,CnForcedLabor,"Changsha, Hunan",China
8,2022-07-14 14:35:04,intent02VAMB,"Toli County Bing Jin Cotton Industry Co., Ltd.",Entity,CnForcedLabor,"Tacheng, Xinjiang",China
9,2022-07-14 14:35:04,intent03amTY,"Korla Transit Station, Agricultural Production...",Entity,CnForcedLabor,,China


In [258]:
final_df_grouped = final_df.groupby(['Date Generated', 'AllimID', 'Name', 'Primary City', 'Primary Country']).agg({
    'Entity Type': list,
    'Risk Labels': list}
).reset_index()[['Date Generated', 'AllimID', 'Name', 'Entity Type', 'Risk Labels', 'Primary City', 'Primary Country']]


In [274]:
def array_to_str(lst):
    if isinstance(lst, list):
        distinct_list = set(lst)
        return '[' + ', '.join(f'"{item}"' for item in distinct_list) + ']'
    return lst

In [275]:
final_df_list_str = final_df_grouped.copy()

final_df_list_str['Entity Type'] = final_df_list_str['Entity Type'].apply(array_to_str)
final_df_list_str['Risk Labels'] = final_df_list_str['Risk Labels'].apply(array_to_str)

# final_df_list_str['Entity Type'] = final_df_list_str['Entity Type'].astype(str)
# final_df_list_str['Risk Labels'] = final_df_list_str['Risk Labels'].astype(str)

In [276]:
final_df_list_str.head(12)

,Date Generated,AllimID,Name,Entity Type,Risk Labels,Primary City,Primary Country
0,2022-07-14 14:35:04,intent018lUW,Aral Southern Xinjiang Hongguopin Famers Profe...,"[""Entity""]","[""CnForcedLabor""]","Alar, Xinjiang",China
1,2022-07-14 14:35:04,intent01F6Yz,"Wusu Yuanda Cotton Industry Co., Ltd.","[""Entity"", ""Vessel""]","[""CnForcedLabor""]","Wusu, Tacheng, Xinjiang",China
2,2022-07-14 14:35:04,intent01GiDN,"Shihezi Tiancheng Water Saving Equipment Co., ...","[""Entity""]","[""CnForcedLabor""]",,China
3,2022-07-14 14:35:04,intent01mMB4,"Xinjiang Xinte Crystal Silicon Hightech Co., Ltd.","[""Entity""]","[""CnForcedLabor""]","Urumqi, Xinjiang",China
4,2022-07-14 14:35:04,intent01sKCr,"Shihezi West Ningxin Carbon Co., Ltd.","[""Entity""]","[""CnForcedLabor""]","Shihezi, Xinjiang",China
5,2022-07-14 14:35:04,intent02CuQE,Hunan Xinxiang Expressway Construction Develop...,"[""Entity""]","[""CnForcedLabor""]","Changsha, Hunan",China
6,2022-07-14 14:35:04,intent02VAMB,"Toli County Bing Jin Cotton Industry Co., Ltd.","[""Entity""]","[""CnForcedLabor""]","Tacheng, Xinjiang",China
7,2022-07-14 14:35:04,intent02arS7,"Yengisar Tianyan Industry Co., Ltd.","[""Entity""]","[""CnForcedLabor""]","Yengisar County, Kashgar Prefecture, Xinjiang",China
8,2022-07-14 14:35:04,intent03amTY,"Korla Transit Station, Agricultural Production...","[""Entity""]","[""CnForcedLabor""]",,China
9,2022-07-14 14:35:04,intent03lyKo,"Xinjiang Longtai Jufeng Concrete Co., Ltd.","[""Entity""]","[""CnForcedLabor"", ""CnRepression""]","Urumqi, Xinjiang",China


In [297]:
final_df_list_str.sort_values(by='AllimID', key=lambda x: x.str.lower()).to_csv('../data/my_output.csv', index=False, lineterminator='')